# **Processing sleeping data**

**1. Combine LFP data from sleeping recordings to the spatial data to account for movement ✔️**


In [ ]:
import sys  
sys.path.insert(0, 'D:/Beths/')
import os
import re
import math
import pandas as pd
import numpy as np
import yasa
import mne
from mne.filter import filter_data, resample
import matplotlib.pyplot as plt
from scipy import signal

## Import from my files
from data_lfp import mne_lfp_Axona, load_lfp_Axona
from data_pos import RecPos

### Sleeping recordings


In [ ]:
df = pd.read_csv('data_scheme.csv', parse_dates=['date_time'])
cols = df.columns
df[cols[2:]].loc[df[cols[2:]].sleep == 1]

In [ ]:
sleep = df.loc[df.sleep == 1]
ax = sleep['rat'].value_counts().plot(kind='bar',
                                    figsize=(9,5),
                                    title="Number of sleep recordings for each animal.")
ax.set_xlabel("Rat ID")
ax.set_ylabel("Frequency of recordings")
plt.show()

In [ ]:
sleep = df.loc[df.sleep == 1]
ax = sleep['treatment'].value_counts().plot(kind='bar',
                                    figsize=(9,5),
                                    title="Treatment number in sleep animals.")
ax.set_xlabel("Treatment type")
ax.set_ylabel("Frequency of recordings")
plt.show()

### **Idea: Use movement as an indicator of awakeness**
#### Open LFP and Spatial information and mark movement as MNE envents)

In [ ]:
def detect_movement(x, y, tresh=2):
    '''function that detects when the animal moved
    
    Inputs:
    x,y (float array): 
        Spatial coordinates x,y
    tresh(float):
        treshold in cm/s 
    Output:
        moving (array):
        1 for movement, 0 for no movement
    '''
    moving = np.zeros(len(x)*5)
    for i in range(1, len(x)):
        if (math.sqrt((x[i] - x[i - 1]) ** 2 + (y[i] - y[i - 1]) ** 2) / (0.02 * 3)) >= tresh:
            moving[5*i:5*i+5] = 1 ## increase sample rate per 5 (size of recording)
    return moving

def create_events(record, events):
    '''Create events on MNE object
    Inputs:
        record(mne_object): recording to add events
        events_time(2D np array): array 0,1 with same lenght of recording dimension (1, lengt(record))
    output: 
    record(mne_object): Record with events added
    '''
    try:
        assert len(record.times) == events.shape[1]
        stim_data = events
        info = mne.create_info(['STI'], record1.info['sfreq'], ['stim'])
        stim_raw = mne.io.RawArray(stim_data, info)
        record.add_channels([stim_raw], force_update_info=True)
    except AssertionError as error:
        print(error)
        print('The lenght of events needs to be equal to record lenght.')
    return record

#### **Open spatial data**

(Spatial data is sampled at 50 Hz, so every 5 LFPs I have one Spatial info)
1. Compare lenghts
2. Syncronize
3. detect movement

Most sleep detection papers use EMG/LFP.

**Detection of slow-wave sleep, sharp-wave ripples** [https://www.cell.com/current-biology/pdfExtended/S0960-9822(19)30006-5]


Mean running speed for each1.6 s bin was also estimated. In the absence of EMG recordings, we could not unequivocally discriminate between slow wavesleep and quiet immmobility, we therefore restricted all analyses to epochs termed ‘rest’. Rest was defined as epochs with runningspeed < 2.5cm/s, and theta/delta power ratio < 2 and waking movement as theta/delta power ratio > 2 and speed > 2.5cm/s. Sharp-wave ripples were detected by first filtering the LFP in the band 100-250Hz. The instantaneous power of the filtered LFP was thenestimated by calculating the root mean square over 7ms intervals (MATLAB function ‘envelope’ with option ‘rms’). From all LFPsacross tetrodes in the CA1 layer, the LFP whose power estimate had the highest standard deviation was then used to define rippleevents, as 100ms windows around the peak power, whenever the power was greater than the 99thpercentile of all powers in the trial(approximately equal to 4 standard deviations above the mean). Multi-unit activity (MUA) bursts were defined by binning all spikesfrom CS cells into 1ms bins and smoothing the resulting binned spike train with a Gaussian kernel (s.d. 10ms). MUA events werethen defined as crossing of a threshold defined as 3 standard deviations above the mean of the smoothed spike train, with a durationfrom 100-750ms. 
Only MUA bursts which temporally overlapped (even in part) with SWR events were included in the replay analysis.

**Spindle detection** [https://www.jneurosci.org/content/jneuro/26/50/12914.full.pdf]

Power spectra of delta(0–4 Hz), theta (5–10 Hz), and spindle (12–15 Hz) frequency bandswere calculated continuously, and sleep–wake episodes were scored byvisual assessment for 10 s epochs according to the standard criteria (Bjor-vatn et al., 1998). Briefly, the awake state was marked by the presence oflow-amplitude fast activity associated with EMG tonus; SWS was identi-fied by continuous high-amplitude slow activity, regular appearance ofspindles, and decreased EMG activity. Transitions from SWS into REMsleep were identified by a decrease in high-amplitude slow activity, in-crease of theta activity, and presence of spindles. REM sleep was charac-terized by dominant theta activity, low-voltage fast activity, and an ab-sence of EMG tonus. Behavioral states were additionally verified by videorecording.

In [ ]:
def plot_treshold(x, y, trehs):
    moving = detect_movement(x, y, trehs)
    ax = plt.figure(figsize=(4,4))
    ax = plt.scatter(x,y, c= 'b', marker='.', alpha = .2)
    x = np.asarray([n*m for n, m in zip(x, moving)])
    y = np.asarray([n*m for n, m in zip(y, moving)])
    x[x==0] = np.nan
    y[y==0] = np.nan
    ax = plt.scatter(x,y, c= 'r', marker='.')
    ax = plt.xlabel('X pixels')
    ax = plt.ylabel('Y pixels')
    plt.tight_layout()
    return plt.show()

In [ ]:
file = sleep_files[7]
pos1 = RecPos(file)
x, y = pos1.get_position()
wview = pos1.get_cam_view()
plot_treshold(x, y, 2)

In [ ]:
for file in sleep_files[10:15]:
    print(file)
    pos1 = RecPos(file)
    x,y = pos1.get_position()
    moving = detect_movement(x, y, 3)
    ax = plt.figure(figsize=(4,4))
    ax = plt.plot(x,y, c= 'b', linewidth=1)
    ax = plt.xlabel('X pixels')
    ax = plt.ylabel('Y pixels')
    plt.tight_layout()
    plt.show()

#### **Select files with sleep**

#### **Open sleep file and add movement as event Channel**

In [ ]:
position = RecPos(file)
x, y = position.get_raw_pos()
moving = detect_movement(x, y, 30)
events = moving.reshape(1, len(moving))
record1 = mne_lfp_Axona(file)
record1 = create_events(record1, events) 

In [ ]:
events = mne.find_events(record1, stim_channel='STI')

In [ ]:
record1.plot(events=events, start=40, duration=600, scalings=dict(eeg=100e-5), event_color={1: 'r'})

In [ ]:
record1['ch_1']

In [ ]:
# Broadband (1 - 30 Hz) bandpass filter
record2 = mne_lfp_Axona(file)
info = record2.info
sf = info['sfreq']
ch = info['ch_names'][0:2]
pick = mne.pick_channels(info['ch_names'], include = info['ch_names'][0:1])
freq_broad = (1, 30)
data_broad = record2.filter(l_freq= freq_broad[0], h_freq=freq_broad[1],  method='fir',verbose=0).copy()

### **Filtering signal into bands**
<ul>
    <li>Delta (1-4Hz)</li>
    <li>Theta (4-12Hz)</li>
    <li>Beta (12-30Hz)</li>
</ul>


In [ ]:
import seaborn as sns
sns.set(font_scale=1.2)

In [ ]:
t0 = 100
tf = 130
rec = record2.filter( l_freq=1, h_freq=30)
data = rec[12][0][0][250*t0:250*tf]
sfreq = 250
print(len(data))

In [ ]:
# Define sampling frequency and time vector
data = signal.resample(data, int(7500/5)) # resampling 
time = np.arange(len(data)) / sfreq
# Plot the signal
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(time, data, lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
# plt.xlim([time.min(), time.max()])
plt.title(f'N3 sleep EEG data {tf-t0}s')
sns.despine()

In [ ]:
info = mne.create_info(ch_names=['RSC'],
                       ch_types=['eeg'],
                       sfreq=250)
ch1 = mne.io.RawArray(record1['ch_1'][0], info)

In [ ]:
# ch1.plot(show_scrollbars=False, show_scalebars=True, scalings=dict(eeg=100e-5))

times = np.arange(len(ch1)) / 250.

# Plot the signal
fig, ax = plt.subplots(1, 1, figsize=(14, 4))
plt.plot(times[12000:12800], record1['ch_1'][0][0][12000:12800], lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (uV)')
# plt.xlim([times.min(), times.max()])
plt.title('N2 sleep EEG data (2 spindles)')
sns.despine()

In [ ]:
from scipy import signal

# Define window length (4 seconds)
win = 4 * sfreq
freqs, psd = signal.welch(data, sfreq, nperseg=win)

# Plot the power spectrum
sns.set(font_scale=1.2, style='white')
plt.figure(figsize=(8, 4))
plt.plot(freqs, psd, color='r', lw=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (V^2 / Hz)')
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram")
plt.xlim([0, freqs.max()])
sns.despine()